# 1 Теория

### 1.1 Введение

Термины "векторизация" или "вложение" (**embedding**) означают представление объекта в виде вектора в неком пространстве. Применяются они, как правило, к нечисловым данным (таким как тексты, графы, последовательности), поскольку числовые данные и так без проблем представляются в виде вектора.

Если мы хотим использовать нечисловые объекты в ML-моделях, такое представление необходимо.

Самое простое векторное представление объекта это **one-hot encoding** (оно же **dummy encoding**). Работает так - мы перечисляем все возможные типы объектов и ставим 1 напротив нашего типа. Например, если мы последовательно кодируем три символа a,b и c, то one-hot encoding для каждого выглядит так:
a=>[1,0,0], b=>[0,1,0], c=>[0,0,1]

One-hot encoding иногда используют в задачах ML, но для текстов или других данных большой размерности оно крайне неээфективно => нужен другой способ кодирования.

**Word2Vec** - алгоритм векторизации слова, при котором часто встречающиеся в одном контексте слова кодируются близкими векторами. Под контекстом понимается 

Разработан в 2013 году Томасом Миколовым. Оригинал статьи: https://arxiv.org/abs/1301.3781

Если получится закодировать слова двумерными векторами, то мы сможем делать красивые визуализации множества слов

<img src="img/wordmap.png" width=500>

Слово, которое нужно векторизовать и его контекст - соседние слова слева и справа. Слово зависит от контекста и наоборот контекст зависит от слова.



Есть две формулировки задачи Word2Vec:

- **Skip-Gram**

  подбираем модель так, чтобы для каждого слова наблюдаемый контекст (скажем 5 соседних слов) был наиболее вероятен


- **CBOW** (continuous Bag-Of-Words)

  подбираем модель так, чтобы для каждого контекста наблюдаемое слово было наиболее вероятным

С точки зрения архитектуры модели всё идентично. Отличаюся способы обучения.
                  
К сожалению в исходной статье алгоритм описан довольно абстрактно и не предлагает ссылок на конекртные реализации. Приходится их додумывать.

Можно с одной матрицей U обучать сеть, тогда на вход идёт пара (слово, другое слово). Как вот тут для CBOW:
Здесь мы просто притягиваем P(word, context_word) к нашим данным. 

Но при таком подходе на выходе только одна вероятность, а если умножать на матрицу V, можем сразу получить распределение веротяностей за один проход по сети.

Схема получения Word2Vec эмбедингов:
Обучаем модель
Отбрасываем правую часть нейронной сети (Head)
Выход Encoder слоя - это и есть эмбединги слова, которое подается на вход. 

Математически задача решается как задача максимизации правдоподобия:

- $J()=w_cP(w|c) \rightarrow max$

- $J()=w_cP(c|w) \rightarrow max$


Обратите внимание, что в рамках модели Word2Vec вложение слова не зависит от соседних слов в кодируемом предложении - оно всгеда одинаково. Это минус, когда мы имеем дело с омонимами (одно написание, но разные смыслы).








Как обучается Word2Vec?



Подбираются такие вектора wi и wj, что веротяность текста максимизируется. В итоге высокая вероятность пары слов наблюдается там, где слова из одного контекста часто встречаются вместе.


# 2 Практика
### 2.1 Реализация в Keras

Попробуем сделать нейронную сеть для построения эмбедингов алгоритмом Word2Vec. Будем использовать API Keras.

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, merge, Dot
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import urllib
import collections
import os
import zipfile

import numpy as np
import tensorflow as tf

Нам нужен большой корпус текстов.

У чувака есть предобработанный датасет с текстами из википедии (без знаков препинания). Весит около 30MB, можно скачать по прямой ссылке:

In [ ]:
!wget 'http://mattmahoney.net/dc/text8.zip'
!unzip -o text8.zip

Обучаться будем на 17 млн слов. 

Небольшой минус данного датасета - текст не делится на предложения и в конец статьи может попасть в начало другой при определении контекстов.

In [ ]:
file=open("text8","r")
words = file.read().split()
file.close()
len(words)

Посчитаем частоту слов. В питоне для этого есть специальный класс collections.Counter.

In [ ]:
import collections
word_freq = collections.Counter(words)

Ожидаемо наиболее частотные слова - артикли и числительные

In [ ]:
word_freq.most_common(10)

Дополним список специальным словом 'UNK' для обозначения более редких слов, чем с частотой n_words

In [ ]:
n_words = 10000
word_freq_list = [['UNK', -1]]
word_freq_list.extend(word_freq.most_common(n_words - 1))

Поставим в соотвествие каждому слову свой порядковый номер

In [ ]:
dictionary = dict()
for word, _ in word_freq_list:
        dictionary[word] = len(dictionary)

Закодируем текст в последовательность индексов слов. Заодно посчитаем частоту группы 'UKN'.

In [ ]:
data = list()
unk_count = 0
for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
count[0][1] = unk_count

Теперь наш текст выглядит так:

In [ ]:
data[0:10]

Потом когда будем анализировать результаты, нам наверняка понадобится обратный словарь (из кода слова в символьное представление).

In [ ]:
reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
reversed_dictionary[1234]

Зададим основные параметры обучения.
- число слов в словаре (vocab_size), мы ранее сами задали ограничение в 10000 слов, поэтому пишем 10000
- размер контекста слова (window_size), сколько сосдених слов будем брать с каждой стороны в качестве контекста слова
- какую хотим получить размерность эмбединга (vector_dim)
- число эпох для обучения нейросети (epochs)

In [ ]:
vocab_size = 10000
window_size = 3
vector_dim = 300
epochs = 200000

In [ ]:
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

Построим частотное распределние слов для Negative Sampling. Для этого в Keras есть специальный метод sequence.make_sampling_table:

In [ ]:
sampling_table = sequence.make_sampling_table(vocab_size)
sampling_table

Также в Keras есть метод для negative-sampling. Он генерирует положительные примеры (слово и слово из контекста) и отрицательные примеры (слово и произвольное слово). Слова выбираются исходя из частот полученных выше.

In [ ]:
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)

Выглядят сэмплы примерно так:

In [ ]:
list(zip(couples[0:10], labels[0:10]))

Разделим наши пары слов на 2 отдельных списка. В питоне для этого есть функция zip(*).

Мы нагенерировали 30 млн пар слов, поэтому обработка займет какое-то время (может 5-10 минут).

In [ ]:
word_target, word_context = zip(*couples)

In [ ]:
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

Теперь будем собирать нейронную сеть. Схема ниже:

<img src="img/word2vec_keras.jpg">

У нас 2 входа сети - один для анализируемого слова (target) и один для слова из его контекста (context). Оба - скаляры (код слова).

In [ ]:
input_target = Input((1,))
input_context = Input((1,))

Далее каждый вход подключаем к эмбедингу (он один и тот же для обоих входов). 

В Keras слой Embedding - это просто словарик с набором описаний, элементы которого сеть считает весами. По мере обучения сети веса выстраиваются так, чтобы достигался минимальный loss.

In [ ]:
embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')

target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)

context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

Перемножаем эмбединги обоих входов. Для скалярного произведения тензоров в Keras сущесвтует слой Dot.

При создании слоя Dot ставим normalize = True. Это означает, что скалряное произведение становится косинусным антирасстоянием (cosine similarity).

In [ ]:
similarity = Dot(axes=0, normalize=True)([target, context])
dot_product = Reshape((1,))(similarity)

У нас softmax бинарная классификация (мы оцениваем вероятность события "слово 1 входит в контекст слова 2").

Поэтому выход определяем как Dense слой со скалярным выходом и сигмоидной активацией.

In [ ]:
output = Dense(1, activation='sigmoid')(dot_product)

Собираем и компилируем модель

In [ ]:
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

Сделаем вариант модели с другой функцией билзости

In [ ]:
validation_model = Model(input=[input_target, input_context], output=similarity)

Пара функций для визуального тестирования результатов

In [104]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reversed_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reversed_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
    
sim_cb = SimilarityCallback()

В каждую эпоху дообучаем сеть на одном примере. Через каждые 10000 примеров для 16 случайно выбранных слов выводим его ближайшие слова.

**Note1:** на ноутбуке сеть может обучаться очень долго. Для целей презентации подхода этого достаточно. Для достижения реального результата очень желательно запускать обучение на мощных серверах либо GPU.

**Note2:** Чтобы сеть обучалась эффективнее, надо все же объединять сэмплы в батчи большего размера.

In [ ]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))

for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    if cnt % 10000 == 0:
        sim_cb.run_sim()

После обучения модели можем выгрузить эмбединги слов:

In [123]:
embeddings = embedding.get_weights()[0]

Сделаем мапинг из слова в эмбединг:

In [132]:
word2embedding = {w:embeddings[idx] for w,idx in dictionary.items()}
word2embedding['love']

### 2.1 Word2Vec в бибилотеке Gensim

In [134]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

ImportError: No module named 'gensim'

Создадим и обучим модель Word2Vec:

In [ ]:
model = Word2Vec(common_texts, 
    size=100, 
    window=5, 
    min_count=1, 
    workers=4)

model.fit(documents, total_examples=len(documents), epochs=10)

In [ ]:
model.vw['love']

In [ ]:
model.save("word2vec.model")

Обученные веса можно сохранить на диск

In [ ]:
model.wv['computer']